# 论文简介
![](https://ai-studio-static-online.cdn.bcebos.com/a7bc44afcfa14e13bcd0ed4045593ad8433aa7a081e74f89b4154ba1fa6d27ae)  
  论文作者提出一种新的、全监督语义分割训练范式，可应用于语义分割的训练中，能够显著提高语义分割的效果。论文在cityscapes验证集上进行实验，HRNet_W48的mIou为81.0%，本次复现赛要求复现的精度为82.2%，本项目复现结果为82.47%。
## 论文核心思想
  作者提出一种新的、全监督语义分割训练范式，利用跨图像的像素-像素之间的关系，来学习一个更好的特征空间。如上图所示，（a）和（b）是训练图片及其对应的标签，传统的语义分割训练时忽略了不同图片之间的上下文信息，而本篇论文提出的跨图像像素对比学习，利用不同图片之间的像素关系，使得特征空间中同种类别的像素相似性变高、不同类别的像素相似度变低（如图d），从而得到一个更好的结构化的特征空间（如图e）。
## 网络结构
![](https://ai-studio-static-online.cdn.bcebos.com/0e7d7d21221f4ed58688cccf39106dc115cb85965cf9448189c686a1ddb1b4bb)  
上图为网络结构，fFCN为backbone模块，fSEG为语义分割head模块，从fSEG平行引出fPROJ模块，fPROJ用来进行对比训练，fSEG训练方法与传统方法相同。
## 损失函数
![](https://ai-studio-static-online.cdn.bcebos.com/3dc4076cf08b46d5b74551491068602b6dabc38a8f44463692026cffae4d2d3e)  
损失函数如上图，由2部分构成，交叉熵损失和对比损失组成，其中对比损失为本篇论文的核心。  

![](https://ai-studio-static-online.cdn.bcebos.com/5b58c4ca0bb24ae1b23fa1e1b713051828bbd0ce4461466baf6c715f81a1ea06)  
交叉熵损失如上图所示，此处不做介绍。  

![](https://ai-studio-static-online.cdn.bcebos.com/7f7cfacb399142a6b0afe127f62f5a2613de0dc069674f86a46f92e0be51ca37)  
上图为对比损失，亦是本篇论文的核心。其中i是真实标签为c的特征向量，i+为正样本像素特征，i-为负样本像素特征，由上式可以看出，通过像素-像素对比学习，在特征空间上同一类别的像素拉近，不同类别的像素原理，从而使得不同类别的像素特征空间能够更好的可区分。   

## 对比损失的anchor采样方法
预测错误的像素被认为是hard anchor，预测正确的像素被认为是easy anchor，在对比度损失计算过程中，一半的anchor是hard anchor，另一半是easy anchor。  

## 实验
![](https://ai-studio-static-online.cdn.bcebos.com/bce32da16bb347bd900ea39d4d03538568d14088286044e8999777e95b96193f)  
上左图为像素交叉熵损失的特征可视图，上右图是对比损失的特征可视化图，可以看出，使用了像素对比损失的语义分割模型特征空间更具结构化。


# 项目介绍
本项目为第四届百度论文复现赛Exploring Cross-Image Pixel Contrast for Semantic Segmentation复现结果，本项目基于PaddleSeg实现，并复现了论文实验结果，非常感谢百度提供的比赛平台以及GPU资源。  

## 复现环境
硬件： Tesla V100 * 4  
框架： paddlepaddle==2.1.2  

## 参数调试
1、project dimension：此参数不影响最后的模型大小，可以适度调整来获得好的效果；  
2、迭代次数：40k次迭代时mIou为81.8%，不满足验收标准，60k次迭代mIou为82.47%，可以适当提高迭代次数；  
3、训练时长：Tesla V100 * 4， batch_size为2，60k次迭代，训练总时长为13.5h。  

## 复现经验
1、使用paddleseg语义分割框架可以有效的减少复现的代码量，提高复现速度；  
2、论文提供的源码使用的框架是pytorch，需要注意不同框架模型转换的细节（尤其是部分函数的不同）。  

## 项目结果

Method|Environment|mIou|Step|Batch_size|Dataset
:--:|:--:|:--:|:--:|:--:|:--:
HRNet_W48_Contrast|Tesla V-100 $\times$ 4 |82.47|60k|2|CityScapes
  

## 项目实现思路
本项目基于PaddleSeg开发，由于PaddleSeg已经包含了数据处理，验证等多项功能，本篇论文的复现仅需要实现HRNet_W48 Head和Contrast loss部分。


## 项目运行实践
1、按照以下步骤可运行本项目。  
2、由于HRNet_W48参数较大，无法上传至Aistudio，可从github的百度云连接下载训练好的权重参数。

In [ ]:
# step 1: git clone, 本项目已clone过，跳过此步骤
!git clone https://hub.fastgit.org/justld/contrast_seg_paddle.git

In [ ]:
!cat model.z* > uns.zip 
!unzip uns.zip

Archive:  uns.zip
warning [uns.zip]:  zipfile claims to be last disk of a multi-part archive;
  attempting to process anyway, assuming all parts have been concatenated
  together in order.  Expect "errors" and warnings...true multi-part support
  doesn't exist yet (coming soon).
warning [uns.zip]:  209715200 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  209715204
  (attempting to re-compensate)
  inflating: model.pdparams          


In [ ]:
unzip -oq /home/aistudio/contrast_seg_paddle/lib.zip

In [ ]:
%cd /home/aistudio
!python /home/aistudio/contrast_seg_paddle/export.py \
    --config /home/aistudio/contrast_seg_paddle/configs/HRNet_W48_contrast_cityscapes_1024x512_60k.yml \
    --model_path model.pdparams \
    --save_dir outputt \
    --input_shape 1 3 1024 512


/home/aistudio
Traceback (most recent call last):
  File "/home/aistudio/contrast_seg_paddle/export.py", line 144, in <module>
    main(args)
  File "/home/aistudio/contrast_seg_paddle/export.py", line 99, in main
    net = cfg.model
  File "/home/aistudio/contrast_seg_paddle/paddleseg/cvlibs/config.py", line 273, in model
    if hasattr(self.train_dataset_class, 'NUM_CLASSES'):
  File "/home/aistudio/contrast_seg_paddle/paddleseg/cvlibs/config.py", line 305, in train_dataset_class
    return self._load_component(dataset_type)
  File "/home/aistudio/contrast_seg_paddle/paddleseg/cvlibs/config.py", line 337, in _load_component
    'The specified component was not found {}.'.format(com_name))
RuntimeError: The specified component was not found Cityscapes.


In [9]:
# step 2: 解压数据集
%cd /home/aistudio/data/data64550
!tar -xf cityscapes.tar
%cd /home/aistudio

/home/aistudio/data/data64550
/home/aistudio


In [2]:
%cd /home/aistudio/data/data156910
!unzip offset_semantic.zip
%cd /home/aistudio

/home/aistudio/data/data156910
Archive:  offset_semantic.zip
   creating: test_offset/semantic/
   creating: test_offset/semantic/offset_hrnext/
  inflating: test_offset/semantic/offset_hrnext/bonn_000021_000019_leftImg8bit.mat  
  inflating: test_offset/semantic/offset_hrnext/berlin_000262_000019_leftImg8bit.mat  
  inflating: test_offset/semantic/offset_hrnext/munich_000239_000019_leftImg8bit.mat  
  inflating: test_offset/semantic/offset_hrnext/munich_000027_000019_leftImg8bit.mat  
  inflating: test_offset/semantic/offset_hrnext/mainz_000000_011879_leftImg8bit.mat  
  inflating: test_offset/semantic/offset_hrnext/bielefeld_000000_048518_leftImg8bit.mat  
  inflating: test_offset/semantic/offset_hrnext/mainz_000001_027377_leftImg8bit.mat  
  inflating: test_offset/semantic/offset_hrnext/mainz_000001_020484_leftImg8bit.mat  
  inflating: test_offset/semantic/offset_hrnext/berlin_000222_000019_leftImg8bit.mat  
  inflating: test_offset/semantic/offset_hrnext/munich_000293_000019_leftI

In [ ]:
# step 3: 训练
# 注意：把configs目录下的HRNet_W48_cityscapes_1024x512_60k中数据集目录更换为自己的数据集目录，如：dataset_root: /home/aistudio/data/data64550/cityscapes
%cd /home/aistudio/contrast_seg_paddle
!python train.py --config configs/HRNet_W48_cityscapes_1024x512_60k.yml  --do_eval --use_vdl --log_iter 100 --save_interval 1000 --save_dir output

In [ ]:
# step 4: 验证
# 注意：由于训练好的参数文件太大，请移步github（https://hub.fastgit.org/justld/contrast_seg_paddle.git）从百度云下载权重文件，并把model_path更换为自己的参数路径
%cd /home/aistudio/contrast_seg_paddle
!python val.py --config configs/HRNet_W48_cityscapes_1024x512_60k.yml --model_path output/best_model/model.pdparams


In [15]:
%cd /home/aistudio/
!python contrast_seg_paddle/predict.py \
       --config /home/aistudio/contrast_seg_paddle/configs/HRNet_W48_cityscapes_1024x512_60k.yml \
       --model_path /home/aistudio/contrast_seg_paddle/output/best_model/model.pdparams \
       --image_path /home/aistudio/data/data64550/cityscapes/val.list \
       --save_dir /home/aistudio/contrast_seg_paddle/output/result


/home/aistudio
2022-07-18 16:37:06 [INFO]	
---------------Config Information---------------
batch_size: 2
iters: 60000
loss:
  coef:
  - 1
  - 0.1
  types:
  - type: CrossEntropyLoss
  - base_temperature: 0.07
    ignore_index: 255
    max_samples: 1024
    max_views: 100
    temperature: 0.1
    type: PixelContrastCrossEntropyLoss
lr_scheduler:
  end_lr: 0.0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  backbone:
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/hrnet_w48_ssld.tar.gz
    type: HRNet_W48
  drop_prob: 0.1
  in_channels: 720
  num_classes: 19
  proj_dim: 720
  type: HRNetW48ContrastHead
optimizer:
  type: sgd
  weight_decay: 0.0002
train_dataset:
  dataset_root: /home/aistudio/data/data64550/cityscapes
  mode: train
  transforms:
  - max_scale_factor: 2.0
    min_scale_factor: 0.5
    scale_step_size: 0.25
    type: ResizeStepScaling
  - crop_size:
    - 1024
    - 512
    type: RandomPaddingCrop
  - type: RandomHorizontalFlip
  - brightness_r

In [22]:
%cd /home/aistudio/contrast_seg_paddle/
!python segfix.py --offset /home/aistudio/data/data156910/val/offset_pred/semantic/offset_hrnext --input /home/aistudio/contrast_seg_paddle/output/result/gray_prediction/leftImg8bit/val/frankfurt --split val

/home/aistudio/contrast_seg_paddle
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
bash: config.profile: 没有那个文件或目录
Saving to /home/aistudio/contrast_seg_paddle/output/label_w_segfix
268 files in total.
/home

In [23]:
%cd /home/aistudio/contrast_seg_paddle/lib/metrics
!python  cityscapes_evaluator.py --gt_dir /home/aistudio/data/data64550/cityscapes/gtFine/val --pred_dir /home/aistudio/contrast_seg_paddle/output/label_w_segfix

/home/aistudio/contrast_seg_paddle/lib/metrics
evaluate the result...
1
Evaluating 1 pairs of images...
Images Processed: 1 

-------------- ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------- 
              |  u   |  e   |  r   |  o   |  s   |  d   |  g   |  r   |  s   |  p   |  r   |  b   |  w   |  f   |  g   |  b   |  t   |  p   |  p   |  t   |  t   |  v   |  t   |  s   |  p   |  r   |  c   |  t   |  b   |  c   |  t   |  t   |  m   |  b   | Prior |
-------------- ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------ ------- 
  ego vehicle | [33;1 0.28   [31;1 0.00   [34;1 0.43   [31;1 0.00   [31;1 0.00 

In [ ]:
%cd /home/aistudio/contrast_seg_paddle/lib/metrics
!python cityeval.py

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 